In [4]:
import pandas as pd

In [5]:
df_com = pd.read_excel('company_list.xlsx', sheet_name = 'stock_category100')
df_com

,시가총액 순위,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,제외,...,세부업종명,결산월,상장주식수,액면가,자본금,대표이사,대표전화,주소,already,연합뉴스에서 url 크롤링된 개수
0,1,5930,삼성전자,KOSPI,전기전자,66000,-700,-1.05,394005648300000,False,...,통신 및 방송 장비 제조업,12,5969782550,100,897513820000,"한종희, 경계현",02-2255-0114,경기도 수원시 영통구 삼성로 129 (매탄동),True,22597
1,2,373220,LG에너지솔루션,KOSPI,전기전자,464000,-3500,-0.75,108576000000000,False,...,일차전지 및 축전지 제조업,12,234000000,500,117000000000,권영수,NaN,"서울특별시 영등포구 여의대로 108, 타워 1",False,2977
2,3,660,SK하이닉스,KOSPI,전기전자,120400,200,0.17,87651484746000,False,...,반도체 제조업,12,728002365,5000,3657652050000,"박정호, 곽노정 (각자대표)",031-630-4114,경기도 이천시 부발읍 경충대로 2091,True,4574
3,4,207940,삼성바이오로직스,KOSPI,의약품,731000,8000,1.11,52028194000000,False,...,기초 의약물질 및 생물학적 제제 제조업,12,71174000,2500,177935000000,임존종보 (Rim John Chongbo),032-455-3114,인천광역시 연수구 송도바이오대로 300,False,2015
4,5,5490,POSCO홀딩스,KOSPI,철강금속,514000,3000,0.59,43469612220000,True,...,1차 철강 제조업,12,84571230,5000,482403125000,"대표이사 회장 최정우, \n대표이사 사장 정기섭",054-220-0114,경상북도 포항시 남구 동해안로 6261 -,True,227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,109,14820,동원시스템즈,KOSPI,화학,30350,1350,4.66,879885408700,False,...,플라스틱제품 제조업,12,28991282,5000,146276965000,"장성학,조점근,서범원",02-589-4700,충청남도 아산시 둔포면 아산밸리동로 100 -,False,46
109,110,192650,드림텍,KOSPI,전기전자,13170,500,3.95,874803619050,False,...,전자부품 제조업,12,66423965,100,6931750900,"김형민, 박찬홍(각자대표)",031-620-1500,경기도 성남시 분당구 황새울로 314,True,9
110,111,10780,아이에스동서,KOSPI,비금속광물,28000,1050,3.90,864992968000,False,...,"시멘트, 석회, 플라스터 및 그 제품 제조업",12,30892606,500,15446303000,"허석헌, 정원호, 김갑진, 이준길",02-3218-6701,서울특별시 강남구 영동대로 741,False,57
111,112,1060,JW중외제약,KOSPI,의약품,37700,1100,3.01,860880857200,False,...,의약품 제조업,12,22835036,2500,58440692500,대표이사 신영섭,02-840-6777,경기도 과천시 과천대로7길 38 -,False,132


In [6]:
company_list = df_com[df_com['제외'] == False].iloc[:100]['종목명']
company_list

0          삼성전자
1      LG에너지솔루션
2        SK하이닉스
3      삼성바이오로직스
5           현대차
         ...   
108      동원시스템즈
109         드림텍
110      아이에스동서
111      JW중외제약
112        대한유화
Name: 종목명, Length: 100, dtype: object

In [79]:
[x for x in company_list if x.startswith('LS')]

['LS ELECTRIC']

In [10]:
import requests
import pandas as pd
from tqdm import tqdm 
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import timedelta, datetime

import urllib.parse

In [12]:
# < headless options >
def headless_option() :
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    return options

In [74]:
def url_crawler(press_code, company_list, headless = False) :

    urls = []
    for press, press_num in press_code.items() :

        for company in tqdm(company_list, desc = f'press : {press}') :

            ds, de = '2021.09.30', '2023.09.30' 

            if len(company) == 1 : com, com_ori = company, company
            else : com, com_ori = company

            de_prev = de
            while(True) : 
                url = f'https://search.naver.com/search.naver?where=news&query={urllib.parse.quote(com)}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds={ds}&de={de}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={press_num}&nso=so%3Add%2Cp%3Afrom{ds.replace(".", "")}to{de.replace(".", "")}&is_sug_officeid=0&office_category=0&service_area=0'

                if headless : driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()), options = headless_option())
                else : driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
                driver.get(url)

                page, url_last = 1, "" 
                while(True) : 

                    url_curr = driver.current_url   
                    if url_last == url_curr : break     # '이전 페이지 = 현재 페이지' 인 경우 종료
                    else : url_last = url_curr      # '이전 페이지 ≠ 현재 페이지' 인 경우

                    # 페이지 html 정보 가져오기
                    s = BeautifulSoup(driver.page_source, 'html.parser')

                    # 개별 뉴스에 대한 접속 url 저장
                    urls.extend([{'press' : press, 'company' : com_ori, 'url' : div.find_all('a', {'class' : 'info'})[-1]['href']} for div in s.find_all('div', {'class' : 'info_group'})])


                    # 다음 페이지로 넘어가는 버튼 클릭
                    try : driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/div[1]/div[2]/div/a[2]').click()
                    except : break

                    if page % 80 != 0 : page += 1
                    else : 
                        de = s.find_all('div', {'class' : 'info_group'})[-1].find_all('span', {'class' : 'info'})[-1].text[:-1]
                        break

                driver.quit()

                if de == de_prev : break
                else : 
                    de_prev = de
    
    df = pd.DataFrame(urls).drop_duplicates()
    
    print(f'\n{len(df)} 개 url 크롤링 완료하였습니다.')

    return df


In [75]:
press_code = {
    '연합뉴스' : 1001,
    '매일경제' : 1009,
    '한국경제' : 1015
}

In [ ]:
# df_urls = url_crawler(press_code, company_list)
# df_urls

In [77]:
## crawling additional

add_company = [
    # [검색할 키워드, 기존 회사명]
    ['에쓰오일', 'S-Oil']
    ]

df_urls = url_crawler(press_code, add_company)
df_urls

press : 연합뉴스:   0%|          | 0/1 [00:00<?, ?it/s]

press : 한국경제: 100%|██████████| 1/1 [00:12<00:00, 12.04s/it]


1497 개 url 크롤링 완료하였습니다.


,press,company,url
0,연합뉴스,S-Oil,https://n.news.naver.com/mnews/article/001/001...
1,연합뉴스,S-Oil,https://n.news.naver.com/mnews/article/001/001...
2,연합뉴스,S-Oil,https://n.news.naver.com/mnews/article/001/001...
3,연합뉴스,S-Oil,https://n.news.naver.com/mnews/article/001/001...
4,연합뉴스,S-Oil,https://n.news.naver.com/mnews/article/001/001...
...,...,...,...
1497,한국경제,S-Oil,https://n.news.naver.com/mnews/article/015/000...
1498,한국경제,S-Oil,https://n.news.naver.com/mnews/article/015/000...
1499,한국경제,S-Oil,https://n.news.naver.com/mnews/article/015/000...
1500,한국경제,S-Oil,https://n.news.naver.com/mnews/article/015/000...


In [18]:
# 로컬에 저장하는 코드
df_urls.to_csv('./url_list.csv', index = False, encoding = 'cp949')

In [78]:
# !scrapy runspider scrapy_news_detail.py